In [1]:
from twitterscraper import query_tweets
from datetime import datetime
from pandas import DataFrame

INFO: {'User-Agent': 'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16'}


## Choose arguments

In [2]:
datetime.today().strftime('%Y-%m-%d')

'2020-04-28'

In [10]:
# advanced query
location =  None

radius = None

terms = [['brood'], ['homemade', 'zelfgemaakt']]
#terms = [['tendeuse', 'tondeuse', 'kaal', 'scheren', 'bald', 'peaky cut']]



# limit (should always be None for our case)
limit = None

## begin and end date are very import to us
begindate = '2020-04-10'
enddate = datetime.today().strftime('%Y-%m-%d')

lang = 'nl'

query = ''
for id1, and_group in enumerate(terms):
    if id1 == 0:
        query += '('
    else:
        query += ' AND ('
    query += ' OR '.join(and_group)
    query += ')'

if location:
    query += ' near:{}'.format(location)
if radius:
    query += ' within:{}'.format(radius)
if lang:
    query += ' lang:{}'.format(lang)
if begindate:
    query += ' since:{}'.format(begindate)
if enddate:
    query += ' until:{}'.format(enddate)

print(query)


(brood) AND (homemade OR zelfgemaakt) lang:nl since:2020-04-10 until:2020-04-28


In [11]:
def valid_date(s):
    try:
        return datetime.strptime(s, "%Y-%m-%d").date()
    except ValueError:
        msg = "Not a valid date: '{0}'.".format(s)
        raise argparse.ArgumentTypeError(msg)

begindate = valid_date(begindate)
enddate = valid_date(enddate)

## Search Tweets

In [12]:
tweets = query_tweets(query= query, limit=limit, begindate=begindate, enddate=enddate, poolsize=1, lang=lang)

INFO: queries: ['(brood) AND (homemade OR zelfgemaakt) lang:nl since:2020-04-10 until:2020-04-28 since:2020-04-10 until:2020-04-28']
INFO: Querying (brood) AND (homemade OR zelfgemaakt) lang:nl since:2020-04-10 until:2020-04-28 since:2020-04-10 until:2020-04-28
INFO: Scraping tweets from https://twitter.com/search?f=tweets&vertical=default&q=(brood)%20AND%20(homemade%20OR%20zelfgemaakt)%20lang%3Anl%20since%3A2020-04-10%20until%3A2020-04-28%20since%3A2020-04-10%20until%3A2020-04-28&l=nl
INFO: Using proxy 190.12.95.170:31260
INFO: Scraping tweets from https://twitter.com/i/search/timeline?f=tweets&vertical=default&include_available_features=1&include_entities=1&reset_error_state=false&src=typd&max_position=TWEET-1250091889111568385-1254793967583088640&q=(brood)%20AND%20(homemade%20OR%20zelfgemaakt)%20lang%3Anl%20since%3A2020-04-10%20until%3A2020-04-28%20since%3A2020-04-10%20until%3A2020-04-28&l=nl
INFO: Using proxy 204.15.243.233:58159
INFO: Scraping tweets from https://twitter.com/i/sea

In [13]:
print(tweets)

[<twitterscraper.tweet.Tweet object at 0x7f32b4fa8780>, <twitterscraper.tweet.Tweet object at 0x7f32b4fa8940>, <twitterscraper.tweet.Tweet object at 0x7f32b4fa8978>, <twitterscraper.tweet.Tweet object at 0x7f32b4fa89e8>, <twitterscraper.tweet.Tweet object at 0x7f32b4fa8b38>, <twitterscraper.tweet.Tweet object at 0x7f32b4fa8b70>, <twitterscraper.tweet.Tweet object at 0x7f32b4fa8ba8>, <twitterscraper.tweet.Tweet object at 0x7f32b4fa8c50>, <twitterscraper.tweet.Tweet object at 0x7f32b4fa8c88>, <twitterscraper.tweet.Tweet object at 0x7f32b4fa8d30>, <twitterscraper.tweet.Tweet object at 0x7f32b4fa8dd8>, <twitterscraper.tweet.Tweet object at 0x7f32b4fa8e48>, <twitterscraper.tweet.Tweet object at 0x7f32b4fa8ef0>, <twitterscraper.tweet.Tweet object at 0x7f32b4fad0b8>, <twitterscraper.tweet.Tweet object at 0x7f32b4fad128>, <twitterscraper.tweet.Tweet object at 0x7f32b4fad198>, <twitterscraper.tweet.Tweet object at 0x7f32b4fad278>, <twitterscraper.tweet.Tweet object at 0x7f32b4fad2e8>]


In [14]:
for tweet in tweets:
    print(tweet.text)

Pindakaas op geroosterd brood blijft mijn favoriet! En deze helemaal  #homemade is the best! #pindakaas #sesam #honing #cocosolie #kokenpic.twitter.com/6gEmB0WVsQ
De lekkerste combinaties op brood - deel 1! | HOMEMADE #1 https://youtu.be/ekAOjiU1z9M  via @YouTube
Nee, zo barbecuen 'wij' ook niet. Wat vlees en vis, maar voornamelijk heerlijke salades, brood en sauzen (zelfgemaakt, geen fles) en natuurlijk een bak vol met zomers fruit. Dat lust jij ook Wen!
Dit weekend zelf pizzabroodjes maken? Zo gepiept! Receptlink in bio! #pizza #pizzabroodjes #zelfbakken #thuisbakken #brood #broodbakken #bakken #ikbakthuis #homemade #hartig #snack @ http://baksels.net  https://www.instagram.com/p/B_Wg5kcjSWT/?igshid=1am1uc6x10kom …
Broodmachine is makkelijk maar ik vind het brood lekkerder zelfgemaakt
De tosti met zelfgemaakt brood is een beeeeeeetje lang pic.twitter.com/jq0wzYcIBt
Feest!
En lekkers zelfgemaakt Turks brood (of een soort naan) pic.twitter.com/GYsxTWGxVF
Vraag me een geschenk te noemen

## Store as dataframe and drop duplicates

In [15]:
import re
def delete_hashtags(tweet):
    return re.sub('#', '', tweet)

tweets_df = DataFrame([[tweet.user_id, delete_hashtags(tweet.text), tweet.retweets, tweet.timestamp] for tweet in tweets], columns=['user_id', 'text', 'retweets', 'timestamp'])

In [16]:
tweets_df

,user_id,text,retweets,timestamp
0,1165190512292372480,Pindakaas op geroosterd brood blijft mijn favo...,0,2020-04-27 15:26:09
1,992354587033198592,De lekkerste combinaties op brood - deel 1! | ...,0,2020-04-27 07:40:23
2,1160831334,"Nee, zo barbecuen 'wij' ook niet. Wat vlees en...",0,2020-04-24 21:19:49
3,547726524,Dit weekend zelf pizzabroodjes maken? Zo gepie...,0,2020-04-24 05:24:54
4,1080051624406630404,Broodmachine is makkelijk maar ik vind het bro...,0,2020-04-22 21:23:37
5,989165840598818816,De tosti met zelfgemaakt brood is een beeeeeee...,0,2020-04-20 14:03:54
6,6360462,Feest!\nEn lekkers zelfgemaakt Turks brood (of...,0,2020-04-19 10:46:27
7,309745298,Vraag me een geschenk te noemen tijdens dit is...,0,2020-04-15 10:49:27
8,1173038461,"Blij dat ik dat ook een beetje kan, het is gee...",0,2020-04-14 16:01:46
9,1165190512292372480,Pindakaas op geroosterd brood blijft mijn favo...,0,2020-04-27 15:26:09


In [17]:
tweets_df = tweets_df.drop_duplicates()

In [18]:
tweets_df

,user_id,text,retweets,timestamp
0,1165190512292372480,Pindakaas op geroosterd brood blijft mijn favo...,0,2020-04-27 15:26:09
1,992354587033198592,De lekkerste combinaties op brood - deel 1! | ...,0,2020-04-27 07:40:23
2,1160831334,"Nee, zo barbecuen 'wij' ook niet. Wat vlees en...",0,2020-04-24 21:19:49
3,547726524,Dit weekend zelf pizzabroodjes maken? Zo gepie...,0,2020-04-24 05:24:54
4,1080051624406630404,Broodmachine is makkelijk maar ik vind het bro...,0,2020-04-22 21:23:37
5,989165840598818816,De tosti met zelfgemaakt brood is een beeeeeee...,0,2020-04-20 14:03:54
6,6360462,Feest!\nEn lekkers zelfgemaakt Turks brood (of...,0,2020-04-19 10:46:27
7,309745298,Vraag me een geschenk te noemen tijdens dit is...,0,2020-04-15 10:49:27
8,1173038461,"Blij dat ik dat ook een beetje kan, het is gee...",0,2020-04-14 16:01:46


## Translate to english for generela sentiment analysis

In [19]:
from googletrans import Translator

In [20]:
translator = Translator()
for text in tweets_df['text']:
    translation = translator.translate(text).text
    print(translation)


Peanut butter on toast is my favorite! And it completely homemade is the best! peanut butter sesame honey coconut oil kokenpic.twitter.com/6gEmB0WVsQ
The best combinations of bread - Part 1! | HOMEMADE 1 https://youtu.be/ekAOjiU1z9M via @YouTube
No, barbecues 'we' do not. What meat and fish, but mostly delicious salads, breads and sauces (homemade, not bottle) and of course a bin full of summer fruits. That lust Wen you too!
This weekend, make your own pizza rolls? Piece of cake! Recipe Link in bio! pizza pizza rolls bake home bread baking bread baking ikbakthuis homemade savory snack @ http://baksels.net https://www.instagram.com/p/B_Wg5kcjSWT/?igshid=1am1uc6x10kom ...
Breadmaker is easy but I find it much better homemade
The sandwich with homemade bread is a long beeeeeeetje pic.twitter.com/jq0wzYcIBt
Party!
And delicious homemade Turkish bread (or any kind of naan) pic.twitter.com/GYsxTWGxVF
Ask them to call me a gift during this isolation and I refer to vol-au-vent with homemade br

## Sentiment analysis

In [21]:
# https://medium.com/@b.terryjack/nlp-pre-trained-sentiment-analysis-1eb52a9d742c
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/arne/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [22]:
# https://towardsdatascience.com/google-translate-api-for-python-723093c2144b
translator = Translator()
for text in tweets_df['text']:
    translation = translator.translate(text).text
    print(translation)
    print('score: {}'.format(sid.polarity_scores(translation)))

Peanut butter on toast is my favorite! And it completely homemade is the best! peanut butter sesame honey coconut oil kokenpic.twitter.com/6gEmB0WVsQ
score: {'neg': 0.0, 'neu': 0.709, 'pos': 0.291, 'compound': 0.8309}
The best combinations of bread - Part 1! | HOMEMADE 1 https://youtu.be/ekAOjiU1z9M via @YouTube
score: {'neg': 0.0, 'neu': 0.69, 'pos': 0.31, 'compound': 0.6696}
No, barbecues 'we' do not. What meat and fish, but mostly delicious salads, breads and sauces (homemade, not bottle) and of course a bin full of summer fruits. That lust Wen you too!
score: {'neg': 0.043, 'neu': 0.812, 'pos': 0.145, 'compound': 0.6948}
This weekend, make your own pizza rolls? Piece of cake! Recipe Link in bio! pizza pizza rolls bake home bread baking bread baking ikbakthuis homemade savory snack @ http://baksels.net https://www.instagram.com/p/B_Wg5kcjSWT/?igshid=1am1uc6x10kom ...
score: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Breadmaker is easy but I find it much better homemade
sc